In [1]:
from sklearn import preprocessing
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, BatchNormalization, LSTM
from tensorflow.compat.v1.keras.layers import CuDNNLSTM
from tensorflow.keras.callbacks import TensorBoard, ModelCheckpoint
import numpy as np
import pandas as pd
from collections import deque
import random
import time
import os

In [12]:
SEQ_LEN = 100
future_pre = 1
ratio_to_pre = "BCH-USD"

epochs = 20
b_size = 100
NAME = f"{SEQ_LEN}-SEQ-{future_pre}-pre-{int(time.time())}"

def classify(current, future):
    if float(future) > float(current):
        return 1
    else:
        return 0

main_df = pd.DataFrame()

ratios = ["BTD-USD", "LTC-USD", "ETH-USD", "BCH-USD"]
for ratio in ratios:
    dataset = f"crypto_data/{ratio}.csv"
    
    df = pd.read_csv("crypto_data/LTC-USD.csv", names=["time", "low", "high", "open", "close", "volume"])
    df.rename(columns={"close": f"{ratio}_close", "volume": f"{ratio}_vloume"}, inplace=True)
    
    df.set_index("time", inplace=True)
    df = df[[f"{ratio}_close", f"{ratio}_vloume"]]
    
    if len(main_df) == 0:
        main_df = df
    else:
        main_df = main_df.join(df)
        
        
main_df['future'] = main_df[f"{ratio_to_pre}_close"].shift(-future_pre)
main_df['target'] = list(map(classify, main_df[f"{ratio_to_pre}_close"], main_df["future"]))

In [13]:
#preprocess
def preprocess_df(df):
    df = df.drop('future', 1)

    for col in df.columns:
        if col != "target":
            df[col] = df[col].pct_change()
            df.dropna(inplace=True)
            df[col] = preprocessing.scale(df[col].values)
            
    df.dropna(inplace=True)
    
    sequential_data = []
    prev_days = deque(maxlen=SEQ_LEN)
    for i in df.values:
        prev_days.append([n for n in i[:-1]])
        if len(prev_days) == SEQ_LEN:
            sequential_data.append([np.array(prev_days), i[-1]])

    #balance
    buys = []
    sells = []
    
    for seq, target in sequential_data:
        if target == 0:
            sells.append([seq, target])
        elif target == 1:
            buys.append([seq, target])
            
    random.shuffle(buys)
    random.shuffle(sells)

    lower = min(len(buys), len(sells))
    
    buys = buys[:lower]
    sells = sells[:lower]
    
    sequential_data = buys+sells
    
    random.shuffle(sequential_data)
    
    x = []
    y = []
    
    for seq, target in sequential_data:
        x.append(seq)
        y.append(target)
        
    return np.array(x), np.array(y)

In [14]:
#create normalized and ready-to-train/test data
times = sorted(main_df.index.values)
last_5pct = times[-int(0.05*len(times))]

validation_main_df = main_df[(main_df.index >= last_5pct)]
main_df = main_df[(main_df.index < last_5pct)]

train_x, train_y = preprocess_df(main_df)
validation_x, validation_y = preprocess_df(validation_main_df)

In [15]:
gpu_options = tf.compat.v1.GPUOptions(per_process_gpu_memory_fraction=0.4)
sess = tf.compat.v1.Session(config=tf.compat.v1.ConfigProto(gpu_options=gpu_options))

model = Sequential()

model.add(CuDNNLSTM(128, input_shape=(train_x.shape[1:]), return_sequences=True))
model.add(Dropout(0.2))
model.add(BatchNormalization())

model.add(CuDNNLSTM(128, return_sequences=True))
model.add(Dropout(0.2)) #0.1
model.add(BatchNormalization())

model.add(CuDNNLSTM(128))
model.add(Dropout(0.2))
model.add(BatchNormalization())

model.add(Dense(64, activation="relu"))
model.add(Dropout(0.2))

model.add(Dense(2, activation="softmax"))

opt = tf.keras.optimizers.Adam(lr=0.001, decay=1e-6)
model.compile(loss='sparse_categorical_crossentropy', optimizer=opt, metrics=['accuracy'])

checkpoint_filepath = './models/BCH-USD/cp-{epoch:02d}.ckpt'

model_checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(filepath=checkpoint_filepath,
                                                               save_weights_only=True,
                                                               monitor='val_accuracy',
                                                               mode='max',
                                                               save_best_only=True)

model.fit(train_x, train_y, batch_size=b_size, epochs=epochs, validation_data=(validation_x, validation_y), callbacks=[model_checkpoint_callback])

Train on 70870 samples, validate on 3318 samples
Epoch 1/20
70870/70870 [==============================] - 34s 478us/sample - loss: 0.7182 - accuracy: 0.5131 - val_loss: 0.6899 - val_accuracy: 0.5356
Epoch 2/20
70870/70870 [==============================] - 30s 427us/sample - loss: 0.6902 - accuracy: 0.5347 - val_loss: 0.6857 - val_accuracy: 0.5561
Epoch 3/20
70870/70870 [==============================] - 30s 429us/sample - loss: 0.6822 - accuracy: 0.5562 - val_loss: 0.6637 - val_accuracy: 0.6052
Epoch 4/20
70870/70870 [==============================] - 30s 429us/sample - loss: 0.6703 - accuracy: 0.5826 - val_loss: 0.6464 - val_accuracy: 0.6239
Epoch 5/20
70870/70870 [==============================] - 30s 422us/sample - loss: 0.6642 - accuracy: 0.5950 - val_loss: 0.6461 - val_accuracy: 0.6290
Epoch 6/20
70870/70870 [==============================] - 30s 420us/sample - loss: 0.6611 - accuracy: 0.5993 - val_loss: 0.6555 - val_accuracy: 0.6242
Epoch 7/20
70870/70870 [=====================

In [16]:
# Loads the weights
checkpoint_path = './models/BCH-USD/cp-17.ckpt'
model.load_weights(checkpoint_path)

# Re-evaluate the model
loss,acc = model.evaluate(validation_x, validation_y, verbose=2)
print("Restored model, accuracy: {:5.2f}%".format(100*acc))

3318/3318 - 1s - loss: 0.6474 - accuracy: 0.6531
Restored model, accuracy: 65.31%
